## ANOVA

`En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.`

Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.



In [2]:
# Tratamiento de datos
import pandas as pd
import numpy as np

#Gráficas
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,8) 

# Test estadísticos
import scipy.stats as stats
from scipy.stats import levene

#Test ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA

In [3]:
# Cargamos nuestro dataframe con los datos estandarizados y normalizados
df = pd.read_csv('../Datos/vinos_norm_estandar.csv', index_col = 0)
df.head(2)

,id,year_birth,education,marital_status,income,kidhome,teenhome,recency,mntwines,mntfruits,...,mntsweetproducts,mntgoldprods,numdealspurchases,numwebpurchases,numcatalogpurchases,numstorepurchases,numwebvisitsmonth,mnt_total_sin_vino,mntwines_sin_cero,mnt_wines_box
0,5524,1957,Graduation,Single,0.235696,0,0,0.307039,635,1.551577,...,1.476500,0.843207,0.349414,1.409304,2.51089,-0.550785,0.693904,2.011116,635.0,14.026740
1,2174,1954,Graduation,Single,-0.235454,1,1,-0.383664,11,-0.636301,...,-0.631503,-0.729006,-0.168236,-1.110409,-0.56872,-1.166125,-0.130463,-0.845274,11.0,3.141178


> Recordamos en la comprobación que hicimos de las asunciones para el ANOVA que:
> * No logramos alcanzar la normalidad de la variable respuesta
> * Varias de nuestras variables predictoras son dependientes entre sí
> * Solo la variable `education` es homocedástica en relación con la variable respuesta
>
> Sin embargo queremos probar de nuevo la homocedasticidad ahora que nuestra variable respueta está más normalizada.

In [4]:
#Realizamos el Test de Levene para las columnas numéricas
numericas= df.select_dtypes(np.number)

In [5]:
for col in numericas.columns:
    _, p_valor = stats.levene(df[col], df['mnt_wines_box'])
    print('homocedasticidad entre', col, 'y mnt_wines_box: p-valor = ', p_valor)

homocedasticidad entre id y mnt_wines_box: p-valor =  0.0
homocedasticidad entre year_birth y mnt_wines_box: p-valor =  1.4169744995337058e-239
homocedasticidad entre income y mnt_wines_box: p-valor =  0.0
homocedasticidad entre kidhome y mnt_wines_box: p-valor =  0.0
homocedasticidad entre teenhome y mnt_wines_box: p-valor =  0.0
homocedasticidad entre recency y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntwines y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntfruits y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntmeatproducts y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntfishproducts y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntsweetproducts y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntgoldprods y mnt_wines_box: p-valor =  0.0
homocedasticidad entre numdealspurchases y mnt_wines_box: p-valor =  0.0
homocedasticidad entre numwebpurchases y mnt_wines_box: p-valor =  0.0
homocedasticidad entre numcatalogpurchases y mnt_

> Aclaramos de nueva cuenta que no tenemos homocedasticidad.
> 
> Sin embargo intentaremos con el test de ANOVA.

In [6]:
# Iniciamos nuestro ANOVA. La variable respuesta es la cantidad gastada en vino normalizada (mnt_wines_box)
lm = ols('mnt_wines_box ~ year_birth  + education + marital_status + income + kidhome + teenhome  + recency + mntfruits + mntmeatproducts + mntfishproducts + mntsweetproducts + mntgoldprods + numdealspurchases + numwebpurchases + numcatalogpurchases + numstorepurchases + numwebvisitsmonth + mnt_total_sin_vino', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
education,4.0,5594.935721,1398.733930,302.153908,1.480332e-207
marital_status,6.0,164.007639,27.334606,5.904810,3.976752e-06
year_birth,1.0,1520.661153,1520.661153,328.492575,1.450438e-68
income,1.0,18034.445681,18034.445681,3895.793280,0.000000e+00
kidhome,1.0,4929.430140,4929.430140,1064.853401,4.791591e-191
teenhome,1.0,244.190604,244.190604,52.749950,5.222088e-13
recency,1.0,19.158486,19.158486,4.138608,4.203458e-02
mntfruits,1.0,1340.789348,1340.789348,289.636744,3.884628e-61
mntmeatproducts,1.0,1594.621686,1594.621686,344.469498,1.383433e-71
mntfishproducts,1.0,334.108952,334.108952,72.174074,3.560253e-17


> Rechazamos que las predictoras no afectan a la variable respuesta, es decir, sí estarían afectando a nuestra variable respuesta en todos los casos

In [26]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          mnt_wines_box   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     372.9
Date:                Tue, 27 Sep 2022   Prob (F-statistic):               0.00
Time:                        12:15:57   Log-Likelihood:                -4881.6
No. Observations:                2240   AIC:                             9815.
Df Residuals:                    2214   BIC:                             9964.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     26.9255      8.619      3.124      0.002      10.024      43.827
education[T.Basic]            -2.2541      0.336     -6.712      0.000      -2.913      -1.596
education[T.Graduation]        0.9964      0.165      6.023      0.000       0.672       1.321
education[T.Master]            1.8275      0.192      9.542      0.000       1.452       2.203
education[T.PhD]               2.1112      0.187     11.307      0.000       1.745       2.477
marital_status[T.Divorced]     0.8786      1.253      0.701      0.483      -1.579       3.336
marital_status[T.Married]      0.9409      1.247      0.755      0.451      -1.504       3.386
marital_status[T.Single]       1.0490      1.249      0.840      0.401      -1.400       3.498
marital_status[T.Together]     1.0624      1.248      0.851      0.395      -1.385       3.510
marital_status[T.Unknown]      0.6956      1.650      0.422      0.673      -2.540       3.931
marital_status[T.Widow]        0.8841      1.270      0.696      0.487      -1.607       3.375
year_birth                    -0.0102      0.004     -2.361      0.018      -0.019      -0.002
income                         0.8409      0.066     12.718      0.000       0.711       0.971
kidhome                       -1.0088      0.115     -8.758      0.000      -1.235      -0.783
teenhome                       0.5282      0.106      4.999      0.000       0.321       0.735
recency                        0.0603      0.046      1.321      0.187      -0.029       0.150
mntfruits                     -0.0088      0.064     -0.138      0.890      -0.134       0.116
mntmeatproducts                0.3200      0.055      5.778      0.000       0.211       0.429
mntfishproducts                0.0549      0.066      0.828      0.408      -0.075       0.185
mntsweetproducts              -0.1605      0.063     -2.543      0.011      -0.284      -0.037
mntgoldprods                   0.2297      0.055      4.150      0.000       0.121       0.338
numdealspurchases              0.1758      0.058      3.022      0.003       0.062       0.290
numwebpurchases                1.1652      0.062     18.883      0.000       1.044       1.286
numcatalogpurchases            0.8523      0.076     11.247      0.000       0.704       1.001
numstorepurchases              1.5302      0.065     23.406      0.000       1.402       1.658
numwebvisitsmonth              0.2726      0.067      4.040      0.000       0.140       0.405
mnt_total_sin_vino             0.2372      0.033      7.191      0.000       0.173       0.302
==============================================================================
Omnibus:                      847.969   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13974.

> Si nos fijamos en el R cuadrado, deducimos que las variables predictoras explican la variable respuesta en un 80%, que es un valor bastante alto. 
>
> En este caso los p valores son diferentes. Nos dice que las siguientes variables predictoras no afectan a nuestra variable respuesta:
> * marital_status
> * recency
> * mntfruits
> * mntfishproducts
>
> Y que la variable predictora que más influye que esté fuera del intercept es:
> * education[T.Basic] con un coeficiente de -2.2541, es decir, tiene una relación inversa
> * education[T.PhD] con un coeficiente de 2.1112, es decir, tiene una relación directa
> 
> Estos resultados dan pie a que hagamos interpretaciones, pero en realidad no son muy fiables, ya que hemos ejecutado el test sin haber cumplido ninguna de las asunciones.